In [ ]:
# Auto-configure repo path and compute device (GPU/MPS/CPU)
import sys
from pathlib import Path

try:
    from utils.path_helpers import add_repo_root_to_sys_path
except Exception:
    cur = Path.cwd()
    for parent in [cur] + list(cur.parents):
        if (parent / "requirements.txt").exists() or (parent / ".git").exists():
            sys.path.insert(0, str(parent))
            break
    from utils.path_helpers import add_repo_root_to_sys_path

add_repo_root_to_sys_path()

from utils.device import get_device, backend_info, backend_name, ensure_seed
print(f"Using backend: {backend_info()}")
ensure_seed(42)

# For PyTorch 2.x, set default device so tensors/models go there automatically
try:
    import torch  # noqa: F401
    if backend_name() in ("torch_cuda", "torch_mps") and hasattr(torch, "set_default_device"):
        torch.set_default_device("cuda" if backend_name() == "torch_cuda" else "mps")
        print(f"torch default device set to {torch.get_default_device()}")
except Exception:
    pass

# Project 14: Pretraining a Tiny Transformer from Scratch

## CORE PROJECT - Understanding Base Models

## Goal
Pretrain a small transformer and watch it learn language patterns.

## Learning Objectives
- What happens during pretraining
- Next-token prediction loss
- Loss dynamics and convergence
- Text generation from trained model
- Why pretraining is expensive but powerful

## Model Configuration
```
Vocabulary: 5000-10000 tokens
Dimension: 384
Heads: 6
Layers: 4-6
Parameters: ~20-50M
Dataset: Shakespeare (~5MB)
Training time on M4: 4-12 hours
```

In [1]:
# Setup
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Training on: {device}")

Training on: mps
